Inference\
https://wikidocs.net/115055

https://huggingface.co/docs/transformers/index

http://kkma.snu.ac.kr/documents/index.jsp?doc=postag


# Author: Yoonhyuck WOO / JBNU_Industrial Information system Engineering
# Date; 2. 10. 2022 - 2. 22. 2022
# Title: Korean_NER
# Professor: Seung-Hoon Na

In [3]:
import os
import json
import numpy as np
from functools import partial
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import BertTokenizer, BertModel, FlaubertModel
# from transformers import BertModel
from sklearn.metrics import classification_report

In [10]:
PATH_dir = 'C:\\Users\\LG\\Desktop\\github\\JBNU-2022-SPRING\\English world class tagging & Korean_Named Entity Recognition\\Ko_En_NER_POStag_data\Ko_NER_POS'
file_name_ko_train = 'train.txt'
file_name_ko_test = 'test.txt'
file_name_ko_dev = 'dev.txt'

In [11]:
file_name = file_name_ko_dev
PATH_data = os.path.join(PATH_dir, file_name)

# change file extention from '.txt' to '.json'
prepro_file_name = 'prepro_' + file_name.rsplit('.')[0] + '.json'
PATH_preprodata = os.path.join(PATH_dir, prepro_file_name)

In [12]:
prepro_tag_name = 'prepro_' + file_name.rsplit('.')[0] + '_tag.json'
PATH_preprotag = os.path.join(PATH_dir, prepro_tag_name)

In [13]:
print(PATH_data)

C:\Users\LG\Desktop\github\JBNU-2022-SPRING\English world class tagging & Korean_Named Entity Recognition\Ko_En_NER_POStag_data\Ko_NER_POS\dev.txt


In [14]:
print(file_name.rsplit('.')[1])

txt


In [15]:
print(prepro_file_name)

prepro_dev.json


In [16]:
print(PATH_preprodata)

C:\Users\LG\Desktop\github\JBNU-2022-SPRING\English world class tagging & Korean_Named Entity Recognition\Ko_En_NER_POStag_data\Ko_NER_POS\prepro_dev.json


In [17]:
with open(PATH_data, 'r', encoding='UTF8') as f:
    rawdata = f.readlines()

In [18]:
df0 = pd.DataFrame(rawdata)
df0

,0
0,선방/NNG\tNNG\tB\tO\n
1,한/XSV~ETM\tXSV~ETM\tI\tO\n
2,정성룡/NNP\tNNP\tB\tB-PS\n
3,이/JKS\tJKS\tI\tO\n
4,볼/NNG\tNNG\tB\tO\n
...,...
6940,문화/NNG\tNNG\tB\tO\n
6941,체육/NNG\tNNG\tI\tO\n
6942,센터/NNG\tNNG\tI\tO\n
6943,)/SS\tSS\tI\tO\n


In [19]:
df = pd.DataFrame(df0[0].str.split('/').tolist(),columns=['1-1','1-2'])
df

,1-1,1-2
0,선방,NNG\tNNG\tB\tO\n
1,한,XSV~ETM\tXSV~ETM\tI\tO\n
2,정성룡,NNP\tNNP\tB\tB-PS\n
3,이,JKS\tJKS\tI\tO\n
4,볼,NNG\tNNG\tB\tO\n
...,...,...
6940,문화,NNG\tNNG\tB\tO\n
6941,체육,NNG\tNNG\tI\tO\n
6942,센터,NNG\tNNG\tI\tO\n
6943,),SS\tSS\tI\tO\n


In [20]:
df2 = pd.DataFrame(df0[0].str.split('\t').tolist(),columns=['1','2','3','4'])
df2

,1,2,3,4
0,선방/NNG,NNG,B,O\n
1,한/XSV~ETM,XSV~ETM,I,O\n
2,정성룡/NNP,NNP,B,B-PS\n
3,이/JKS,JKS,I,O\n
4,볼/NNG,NNG,B,O\n
...,...,...,...,...
6940,문화/NNG,NNG,B,O\n
6941,체육/NNG,NNG,I,O\n
6942,센터/NNG,NNG,I,O\n
6943,)/SS,SS,I,O\n


In [21]:
df7 = pd.concat([df['1-1'],df2['4'].str.strip('\n')],axis = 1)
df7 = pd.DataFrame(df7)
df7 = df7.rename(columns={'1-1':'Entity','4':'tag'})
df7

,Entity,tag
0,선방,O
1,한,O
2,정성룡,B-PS
3,이,O
4,볼,O
...,...,...
6940,문화,O
6941,체육,O
6942,센터,O
6943,),O


In [22]:
print(len(df7))

6945


In [23]:
df7['tag'].unique()

array(['O', 'B-PS', None, 'B-DT', 'I-DT', 'B-OG', 'B-TI', 'I-TI', 'I-PS',
       'I-OG', 'B-LC', 'I-LC', 'B-', 'I-'], dtype=object)

# Preprocessing 1

In [24]:
lst=[]
lst_set=[]
lst_ent=[]
lst_tag=[]
for i in range(6945):
    if df7['tag'][i] != None:
        lst_ent.append(df7['Entity'][i])
        lst_tag.append(df7['tag'][i])
    else:
        lst_set.append(lst_ent)
        lst_set.append(lst_tag)
        lst.append(lst_set)
        lst_ent = []
        lst_tag = []
        lst_set = [] 

In [25]:
print(lst[0][0])
print(lst[0][1])

['선방', '한', '정성룡', '이', '볼', '을', '걷', '어', '내', '고', '있', '다', '.']
['O', 'O', 'B-PS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [26]:
print(len(df7['Entity'][0]))
print(df7['Entity'][0])
morph_to_tokens = tokenizer.tokenize(df7['Entity'][0])
print(morph_to_tokens)
print(len(morph_to_tokens))

2
선방


NameError: name 'tokenizer' is not defined

# Preprocessing 2

만약, BERT의 단어 집합에 em, ##bed, ##ding, #s라는 서브 워드들이 존재한다면, embeddings는 em, ##bed, ##ding, #s로 분리됩니다. 여기서 ##은 이 서브워드들은 단어의 중간부터 등장하는 서브워드라는 것을 알려주기 위해 단어 집합 생성 시 표시해둔 기호입니다. 이런 표시가 있어야만 em, ##bed, ##ding, #s를 다시 손쉽게 embeddings로 복원할 수 있을 것

In [29]:
PRETAINED_MODEL_NAME = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(PRETAINED_MODEL_NAME)

In [ ]:
lst=[]
lst_set = []
lst_ent = []
lst_tag = []
for i in range(6945):
    if df7['tag'][i] != None:
        lst_ent.append(df7['Entity'][i])
        lst_tag.append(df7['tag'][i])

    else:
        lst_set.append(lst_ent)
        result = ' '.join(str(s) for s in lst_ent)
        morph_to_tokens = tokenizer.tokenize(result)
        lst_set.append(morph_to_tokens)
        lst_set.append(lst_tag)
        lst.append(lst_set)
        result = 0
        lst_ent = []
        lst_tok = []
        lst_tag = []
        lst_set = [] 

In [ ]:
print('entity', len(lst[0][0]))
print(lst[0][0])

print('token', len(lst[0][1]))
print(lst[0][1])

print('entity_tag', len(lst[0][2]))
print(lst[0][2])

# print('tok_tag', len(lst[3][]))
# print(lst[3][])

In [ ]:
print(len(lst[3][0][2]))
print(lst[3][0])


# Tokenizer tagging

In [34]:
lst=[]
lst_set = []
lst_ent = []
lst_tag = []
for i in range(len(df7)):
    if df7['tag'][i] != None:
        morph_to_tokens = tokenizer.tokenize(df7['Entity'][i])
        if 1 == len(morph_to_tokens):
            lst_ent.append(df7['Entity'][i])
            lst_tag.append(df7['tag'][i])
            
        else:
            if df7['tag'][i] == 'O':
                lst_ent.append(df7['Entity'][i])
                lst_tag.append(df7['tag'][i])
                for i in range(len(morph_to_tokens)-1):
                    lst_tag.append(df7['tag'][i])
                
            else:
                lst_ent.append(df7['Entity'][i])
                lst_tag.append(df7['tag'][i])
                split = df7['tag'][i].split('-')
                for i in range(len(morph_to_tokens)-1):
                    results = 'I-' + split[-1]
                    lst_tag.append(results)
            
    else:
        lst_ent = result = ' '.join(str(s) for s in lst_ent)
        lst_set.append(lst_ent)
        morph_to_tokens = tokenizer.tokenize(lst_ent)
        lst_set.append(morph_to_tokens)
        lst_set.append(lst_tag)
        lst.append(lst_set)
        result = 0
        lst_ent = []
        lst_tok = []
        lst_tag = []
        lst_set = [] 

In [35]:
print('entity', len(lst[3][0]))
print(lst[3][0])

print('token', len(lst[3][1]))
print(lst[3][1])

print('token_tag', len(lst[3][2]))
print(lst[3][2])

entity 139
29 분 마이클 캐릭 의 침투 패스 를 오른쪽 측면 에서 이어받 은 박지성 은 절묘 한 개인기 로 미들즈브러 수비수 를 제친 뒤 골문 으로 강하 고 정확 한 패스 를 넣 었 고 루니 가 이 를 오른발 슛 으로 연결 , 승부 를 원점 으로 돌렸 다 .
token 83
['29', '분', '마', '##이', '##클', '캐', '##릭', '의', '침', '##투', '패', '##스', '를', '오', '##른', '##쪽', '측', '##면', '에서', '이어', '##받', '은', '박', '##지', '##성', '은', '절', '##묘', '한', '개', '##인', '##기', '로', '미', '##들', '##즈', '##브', '##러', '수', '##비', '##수', '를', '제', '##친', '뒤', '골', '##문', '으로', '강', '##하', '고', '정', '##확', '한', '패', '##스', '를', '넣', '었', '고', '루', '##니', '가', '이', '를', '오', '##른', '##발', '슛', '으로', '연', '##결', ',', '승', '##부', '를', '원', '##점', '으로', '돌', '##렸', '다', '.']
token_tag 83
['B-TI', 'I-TI', 'B-PS', 'I-PS', 'I-PS', 'I-PS', 'I-PS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PS', 'I-PS', 'I-PS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-OG', 'I-OG', 'I-OG', 'I-OG', 'I-OG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

In [36]:
print(lst[3])

['29 분 마이클 캐릭 의 침투 패스 를 오른쪽 측면 에서 이어받 은 박지성 은 절묘 한 개인기 로 미들즈브러 수비수 를 제친 뒤 골문 으로 강하 고 정확 한 패스 를 넣 었 고 루니 가 이 를 오른발 슛 으로 연결 , 승부 를 원점 으로 돌렸 다 .', ['29', '분', '마', '##이', '##클', '캐', '##릭', '의', '침', '##투', '패', '##스', '를', '오', '##른', '##쪽', '측', '##면', '에서', '이어', '##받', '은', '박', '##지', '##성', '은', '절', '##묘', '한', '개', '##인', '##기', '로', '미', '##들', '##즈', '##브', '##러', '수', '##비', '##수', '를', '제', '##친', '뒤', '골', '##문', '으로', '강', '##하', '고', '정', '##확', '한', '패', '##스', '를', '넣', '었', '고', '루', '##니', '가', '이', '를', '오', '##른', '##발', '슛', '으로', '연', '##결', ',', '승', '##부', '를', '원', '##점', '으로', '돌', '##렸', '다', '.'], ['B-TI', 'I-TI', 'B-PS', 'I-PS', 'I-PS', 'I-PS', 'I-PS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PS', 'I-PS', 'I-PS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-OG', 'I-OG', 'I-OG', 'I-OG', 'I-OG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PS', 'I-PS', 'O', 'O

In [37]:
with open(PATH_preprodata, 'w') as f:
    json.dump(lst, f)

In [52]:
with open(PATH_preprodata, 'r') as f:
    preprodata = json.load(f)

In [73]:
print(preprodata[0])
print('='*33)
print(preprodata[0][0])
print('='*33)
print(preprodata[0][1])
print('='*33)
print(preprodata[0][2])

['선방 한 정성룡 이 볼 을 걷 어 내 고 있 다 .', ['선', '##방', '한', '정', '##성', '##룡', '이', '볼', '을', '걷', '어', '내', '고', '있', '다', '.'], ['O', 'O', 'O', 'B-PS', 'I-PS', 'I-PS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]
선방 한 정성룡 이 볼 을 걷 어 내 고 있 다 .
['선', '##방', '한', '정', '##성', '##룡', '이', '볼', '을', '걷', '어', '내', '고', '있', '다', '.']
['O', 'O', 'O', 'B-PS', 'I-PS', 'I-PS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [114]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from ner_collate_fn import make_same_len

In [99]:
def collate_fn_custom(data):
    
    input_sent = [sample[0] for sample in data]
    labels = [sample[2] for sample in data]
    
    input_sent = torch.tensor(input_sent)
    
    padded_inputs = torch.nn.utils.rnn.pad_sequence(input_sent, batch_first = True)
    
    return padded_inputs
#     return {'input': padded_inputs.contiguous(),
#             'label': torch.stack(labels).contiguous()}

In [115]:
collate_fn_custom(preprodata)

ValueError: too many dimensions 'str'

In [134]:
def ner_fn_custom(tokenizer, data):
    
    input_sent = [sample[0] for sample in data]
    labels = [sample[2] for sample in data]
    
    batch_inputs = tokenizer(input_sent, padding = True, return_tensors = 'pt')
    batch_labels = make_same_len(labels)
    
#     print('='*33,'batch_inputs[input_ids]','='*33)
#     print(batch_inputs['input_ids'])
    
    
#     print('='*33,'batch_inputs[input_ids].size(1)','='*33)
#     print(batch_inputs['input_ids'].size(1))
    
    
#     print('='*33,'finish','='*33)
    return batch_inputs, batch_labels
#     return {'input': padded_inputs.contiguous(),
#             'label': torch.stack(labels).contiguous()}

In [135]:
ner_fn_custom(tokenizer, preprodata)

each_len_list [16, 20, 57, 83, 34, 66, 44, 43, 4, 61, 65, 76, 117, 50, 50, 38, 29, 55, 54, 21, 60, 25, 88, 58, 17, 18, 37, 25, 18, 23, 43, 13, 40, 31, 40, 21, 16, 31, 23, 11, 18, 8, 38, 126, 25, 44, 34, 33, 47, 24, 32, 25, 21, 22, 54, 54, 42, 26, 60, 59, 13, 35, 11, 21, 59, 90, 41, 26, 92, 7, 54, 25, 32, 33, 35, 35, 30, 130, 24, 30, 67, 59, 22, 31, 27, 24, 19, 25, 44, 14, 36, 36, 32, 20, 56, 50, 22, 32, 16, 40, 58, 17, 21, 30, 20, 44, 69, 51, 24, 29, 11, 49, 43, 23, 42, 25, 33, 54, 35, 42, 84, 4, 82, 105, 50, 37, 35, 39, 64, 8, 13, 17, 26, 29, 48, 14, 69, 18, 19, 28, 34, 11, 54, 57, 93, 61, 103, 63, 50, 99, 28, 20, 15, 90, 46, 64, 28, 27, 36, 42, 8, 46, 95, 26, 21, 36, 42, 29, 42, 126, 51, 40, 39, 68, 23, 36, 11, 21, 20, 54, 44, 18, 24, 22, 16, 55, 26, 113, 13, 49, 56, 20, 47, 19, 47, 33, 18, 27, 34, 20, 24, 12, 32, 63, 46, 26, 95, 37, 69, 28, 22, 10, 12, 10, 29, 37, 41, 27, 48, 30, 47, 63, 22, 73, 61, 27, 47, 10, 18, 34, 69, 38, 50, 24, 38, 41, 31, 24, 25, 19, 35, 43, 50, 12, 30, 23, 

({'input_ids': tensor([[  101,  9428, 42337,  ...,     0,     0,     0],
         [  101,  8935, 23321,  ...,     0,     0,     0],
         [  101,  9144,  9319,  ...,     0,     0,     0],
         ...,
         [  101,  1859,  9767,  ...,     0,     0,     0],
         [  101,  1859, 18778,  ...,     0,     0,     0],
         [  101,  1859,  9665,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])},
 [['O',
   'O',
   'O',
   'B-PS',
   'I-PS',
   'I-PS',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',


In [78]:
input_sent = [sample[0] for sample in preprodata]
labels = [sample[2] for sample in preprodata]

In [82]:
print(input_sent[0:3])

print('='*33)
print(labels[0:3])

['선방 한 정성룡 이 볼 을 걷 어 내 고 있 다 .', '김학범 감독 이 장학영 에게 작전 지시 를 하 고 있 다 .', '또 박지성 은 지난 달 16 일 더비 카운티전 이후 4 경기 만 에 프리미어리그 경기 에 출장 , 지난 달 2 일 풀럼전 골 포함 시즌 두 번 째 공격 포인트 를 기록 했 다 .']
[['O', 'O', 'O', 'B-PS', 'I-PS', 'I-PS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-PS', 'I-PS', 'I-PS', 'O', 'O', 'B-PS', 'I-PS', 'I-PS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'B-PS', 'I-PS', 'I-PS', 'O', 'B-DT', 'I-DT', 'I-DT', 'I-DT', 'I-DT', 'O', 'O', 'O', 'O', 'O', 'B-PS', 'O', 'O', 'O', 'O', 'O', 'B-OG', 'I-OG', 'I-OG', 'I-OG', 'I-OG', 'O', 'O', 'O', 'O', 'O', 'B-DT', 'I-DT', 'I-DT', 'I-DT', 'I-DT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]


In [ ]:
inputs = [k for sample in data[0][k]]
labels = [v for sample in data[1][v]]
padded_inputs = torch.nn.utils.rnn.pad_sequence(inputs, batch_first = True)

{'input': padded_inputs.contiguous(),
 'label': torch.stack(labels).contiguous()}

In [ ]:
for data in dataloader_dev:
    print(data['input'], data['label'])

# Example

In [39]:
sent_morph = '29 분 마이클 캐릭 의 침투 패스 를 오른쪽 측면 에서 이어받 은 박지성 은 절묘 한 개인기 로 미들즈브러 수비수 를 제친 뒤 골문 으로 강하 고 정확 한 패스 를 넣 었 고 루니 가 이 를 오른발 슛 으로 연결 , 승부 를 원점 으로 돌렸 다 .'
sent = '29분 마이클 캐릭의 침투 패스를 오른쪽 측면에서 이어받은 박지성은 절묘한 개인기로 미들즈브러 수비수를 제친 뒤 골문으로 강하고 정확한 패스를 넣었고 루니가 이를 오른발 슛으로 연결, 승부를 원점으로 돌렸다.'

In [40]:
morph_to_tokens = tokenizer.tokenize(sent)
print(len(morph_to_tokens))
print(morph_to_tokens)

73
['29', '##분', '마', '##이', '##클', '캐', '##릭', '##의', '침', '##투', '패', '##스를', '오', '##른', '##쪽', '측', '##면', '##에서', '이어', '##받', '##은', '박', '##지', '##성은', '절', '##묘', '##한', '개', '##인', '##기로', '미', '##들', '##즈', '##브', '##러', '수', '##비', '##수를', '제', '##친', '뒤', '골', '##문', '##으로', '강', '##하고', '정', '##확', '##한', '패', '##스를', '넣', '##었고', '루', '##니', '##가', '이를', '오', '##른', '##발', '슛', '##으로', '연', '##결', ',', '승', '##부를', '원', '##점', '##으로', '돌', '##렸다', '.']


In [41]:
sent_to_tokens = tokenizer.tokenize(sent_morph)
print(len(sent_to_tokens))
print(sent_to_tokens)

83
['29', '분', '마', '##이', '##클', '캐', '##릭', '의', '침', '##투', '패', '##스', '를', '오', '##른', '##쪽', '측', '##면', '에서', '이어', '##받', '은', '박', '##지', '##성', '은', '절', '##묘', '한', '개', '##인', '##기', '로', '미', '##들', '##즈', '##브', '##러', '수', '##비', '##수', '를', '제', '##친', '뒤', '골', '##문', '으로', '강', '##하', '고', '정', '##확', '한', '패', '##스', '를', '넣', '었', '고', '루', '##니', '가', '이', '를', '오', '##른', '##발', '슛', '으로', '연', '##결', ',', '승', '##부', '를', '원', '##점', '으로', '돌', '##렸', '다', '.']


In [42]:
for j in range(7):
    set1 = set(lst[j][2])
    print(set1)

{'O', 'B-PS', 'I-PS'}
{'O', 'I-PS', 'B-PS'}
{'B-PS', 'B-DT', 'I-DT', 'O', 'B-OG', 'I-OG', 'I-PS'}
{'B-TI', 'B-PS', 'O', 'B-OG', 'I-OG', 'I-TI', 'I-PS'}
{'B-PS', 'B-DT', 'O', 'B-OG', 'I-OG'}
{'I-LC', 'B-PS', 'O', 'B-OG', 'B-LC', 'I-OG', 'I-PS'}
{'B-PS', 'B-DT', 'I-DT', 'O', 'B-OG', 'I-OG'}


In [43]:
new_lst = []
set1 = set(lst[5][2])
set2 = set(lst[6][2])
temp = set1.union(set2)
new_lst = temp
    
print(new_lst)

{'I-LC', 'B-PS', 'B-DT', 'I-DT', 'O', 'B-OG', 'B-LC', 'I-OG', 'I-PS'}


In [44]:
new_lst = []
for j in range(7):
    set1 = set(lst[j][2])
    set2 = set(lst[j+1][2])
    print(j, j+1)
    temp = set1.union(set2)
    new_lst = temp
    j = j+2
    
print(new_lst)

0 1
1 2
2 3
3 4
4 5
5 6
6 7
{'B-PS', 'B-DT', 'I-DT', 'O', 'B-OG', 'I-OG', 'I-PS'}


In [31]:
new_lst = []
for j in range(250):
    if j == 0:
        set1 = set(lst[j][2])
        set2 = set(lst[j+1][2])
        temp = set1.union(set2)
        new_lst = temp
    else:
        set1 = set(new_lst)
        set2 = set(lst[j][2])
        temp = set1.union(set2)
        new_lst = temp

new = list(new_lst)

# new.sort()
new.reverse()

# 순서 앞뒤로바꾸고
# 알파벳 순
# dictionary = {string : i for string, i in enumerate(new)}
# new_lst.append('[PAD])')
print (new)
print(dictionary)

IndexError: list index out of range

In [32]:
with open(PATH_preprotag, 'w') as f:
    json.dump(new, f)

NameError: name 'new' is not defined

In [33]:
with open(PATH_preprotag, 'r') as f:
    preprotag = json.load(f)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

# Function Tag_ID

In [58]:
def Tag_ID_Converter():
    new_lst = []
    for j in range(250):
        if j == 0:
            set1 = set(lst[j][2])
            set2 = set(lst[j+1][2])
            temp = set1.union(set2)
            new_lst = temp
        else:
            set1 = set(new_lst)
            set2 = set(lst[j][2])
            temp = set1.union(set2)
            new_lst = temp

    new = list(new_lst)

    new.append('[PAD]')
    # new.sort()
    new.reverse()
#     new.remove('B-')
#     new.remove('I-')
    # 순서 앞뒤로바꾸고
    # 알파벳 순
#     dictionary = {string : i for string, i in enumerate(new)}
    # new_lst.append('[PAD])')

# My Data

In [65]:
result = ' '.join(str(s) for s in lst[3][0])
print(result)

2 9   분   마 이 클   캐 릭   의   침 투   패 스   를   오 른 쪽   측 면   에 서   이 어 받   은   박 지 성   은   절 묘   한   개 인 기   로   미 들 즈 브 러   수 비 수   를   제 친   뒤   골 문   으 로   강 하   고   정 확   한   패 스   를   넣   었   고   루 니   가   이   를   오 른 발   슛   으 로   연 결   ,   승 부   를   원 점   으 로   돌 렸   다   .


In [39]:
inputs = tokenizer(result, return_tensors='pt')
print(inputs['input_ids'].size())
inputs

torch.Size([1, 91])


{'input_ids': tensor([[ 101,  123,  130, 9367, 9246, 9638, 9836, 9792, 9239, 9637, 9783, 9881,
         9909, 9477, 9233, 9580, 9232, 9722, 9776, 9279, 9559, 9425, 9638, 9546,
         9322, 9632, 9319, 9706, 9434, 9632, 9666, 9293, 9954, 8857, 9640, 8932,
         9202, 9309, 9117, 9700, 9376, 9176, 9460, 9379, 9460, 9233, 9672, 9781,
         9109, 8892, 9297, 9630, 9202, 8853, 9952, 8888, 9670, 9994, 9954, 9909,
         9477, 9233, 9010, 9557, 8888, 9213, 9049, 8843, 9638, 9233, 9580, 9232,
         9323, 9476, 9630, 9202, 9568, 8881,  117, 9484, 9365, 9233, 9612, 9668,
         9630, 9202, 9091, 9199, 9056,  119,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,

In [40]:
from transformers import BertModel

In [41]:
PRETAINED_MODEL_NAME = 'bert-base-multilingual-cased'
bert = BertModel.from_pretrained(PRETAINED_MODEL_NAME)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [50]:
outputs = bert(**inputs)

In [94]:
type(bert)

transformers.models.bert.modeling_bert.BertModel

In [51]:
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [52]:
outputs

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.2121, -0.0035, -0.4142,  ...,  0.0273,  0.2088, -0.0627],
         [ 0.2405, -0.1047,  0.4822,  ...,  0.5600, -0.0123,  0.3117],
         [ 0.2475,  0.1201,  0.3328,  ...,  0.4195,  0.3644,  0.2941],
         ...,
         [ 0.0769, -0.2160,  0.1733,  ..., -0.1435,  0.5693, -0.3862],
         [ 0.2981, -0.1335, -0.4855,  ...,  0.2712,  0.2321, -0.0398],
         [ 0.4293, -0.0479, -0.3347,  ...,  0.0816,  0.3221, -0.0263]]],
       grad_fn=<NativeLayerNormBackward>), pooler_output=tensor([[ 1.7947e-01,  5.6591e-02,  1.7773e-01, -1.9114e-01, -9.3124e-02,
          3.4913e-01,  8.9631e-02,  2.0517e-01, -3.3357e-01,  1.9782e-01,
         -1.3146e-02, -1.4954e-01, -1.7576e-01, -8.2157e-02,  8.7189e-02,
         -1.0694e-01,  4.5290e-01,  6.9730e-02,  4.1612e-02, -1.4175e-01,
         -9.9992e-01, -6.7358e-02, -1.1541e-01, -6.4596e-02, -1.6569e-01,
          3.5582e-04, -1.3665e-01,  5.1037e-02, -3.9361e-02, -1.1572

In [104]:
class Bert_NER(nn.Module):
    
    def __init__(self, bert):
        super().__init__()
        bert = BertModel.from_pretrained(PRETAINED_MODEL_NAME)
        self.bert = bert
        self.dorpout = nn.Dropout(p = 0.1)
        self.linear = nn.Linear(768, 22)
        self.softmax = nn.Softmax(dim = 2) # A dimension along which Softmax will be computed
        
    
    def forward(self, **kwargs):
        print(type(self.bert))
        emb = self.bert(**kwargs)
        print(type(self.bert))
        e = self.dropout(emb['last_hidden_state'])
        w = self.linear(e)
        
        return w

In [105]:
model = Bert_NER(22)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [106]:
type(model.bert)

transformers.models.bert.modeling_bert.BertModel

In [107]:
model.forward()

<class 'transformers.models.bert.modeling_bert.BertModel'>


ValueError: You have to specify either input_ids or inputs_embeds